In [4]:

from langgraph.graph import StateGraph, END
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import BedrockChat
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain_core.runnables import RunnableLambda
import json, re 


ModuleNotFoundError: No module named 'langgraph.prebuilt.tool_executor'

In [1]:


# ---------------- TOOLS ----------------
@tool
def get_capital(country: str) -> str:
    capitals = {"France": "Paris", "Germany": "Berlin"}
    return capitals.get(country, "Unknown")

@tool
def get_population(country: str) -> str:
    pops = {"France": 68000000, "Germany": 83000000}
    return str(pops.get(country, 0))

@tool
def calculate_difference(values: str) -> str:
    a, b = map(int, values.split(","))
    return str(abs(a - b))

tools = [get_capital, get_population, calculate_difference]
tool_executor = ToolExecutor(tools)

# ---------------- PLANNER ----------------
# planner_llm = ChatOpenAI(model="gpt-4", temperature=0)
planner_llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or another model like "ai21.j2-ultra" etc.
    region_name="us-east-1",  # or your region
    temperature=0,
    max_tokens=2048
)

def planner_agent(state):
    query = state["query"]
    prompt = f"""
You are a planner. Break down the query into high-level subtasks with executor type.
Query: {query}
Respond in JSON list like:
[
  {{"executor": "researcher", "task": "Find Tesla's recent performance."}},
  {{"executor": "cot", "task": "Get capital of France."}},
  {{"executor": "math", "task": "Calculate revenue change from Q1 to Q2."}}
]
"""
    response = planner_llm.invoke(prompt)
    subtasks = json.loads(response.content)
    return {"tasks": subtasks, "current_step": 0, "results": [], "query": query}

# ---------------- EXECUTORS ----------------
# llm = ChatOpenAI(model="gpt-4", temperature=0)
llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or another model like "ai21.j2-ultra" etc.
    region_name="us-east-1",  # or your region
    temperature=0,
    max_tokens=2048
)

REACT_PROMPT = """
You are a chain-of-thought executor with tools.
Task: {task}
Available tools: get_capital, get_population, calculate_difference
Use this format:
Thought: ...
Action: ...
Action Input: ...
Observation: ...
Final Answer: ...
"""

def cot_executor(state):
    task = state["tasks"][state["current_step"]]["task"]
    scratchpad = ""
    for _ in range(5):
        prompt = REACT_PROMPT.format(task=task) + scratchpad
        output = llm.invoke(prompt).content.strip()
        if "Final Answer:" in output:
            answer = output.split("Final Answer:")[1].strip()
            return {**state, "current_step": state["current_step"] + 1, "results": state["results"] + [answer]}
        m = re.search(r"Action: (\w+)\s*Action Input: (.+)", output)
        if m:
            tool, tool_input = m.group(1).strip(), m.group(2).strip()
            result = tool_executor.invoke([ToolInvocation(tool, tool_input)])[0]
            scratchpad += f"{output}\nObservation: {result}\n"
    return state

RESEARCH_PROMPT = """
You are a researcher. You should look up or synthesize knowledge from known facts.
Task: {task}
Respond with a summary of your findings.
"""

def researcher_executor(state):
    task = state["tasks"][state["current_step"]]["task"]
    response = llm.invoke(RESEARCH_PROMPT.format(task=task)).content.strip()
    return {**state, "current_step": state["current_step"] + 1, "results": state["results"] + [response]}

MATH_PROMPT = """
You are a math-focused executor. Solve the task directly.
Task: {task}
"""

def math_executor(state):
    task = state["tasks"][state["current_step"]]["task"]
    response = llm.invoke(MATH_PROMPT.format(task=task)).content.strip()
    return {**state, "current_step": state["current_step"] + 1, "results": state["results"] + [response]}

# ---------------- ROUTER ----------------
def route(state):
    executor_type = state["tasks"][state["current_step"]]["executor"]
    return executor_type

# ---------------- GRAPH ----------------
graph = StateGraph()

graph.add_node("plan", planner_agent)
graph.add_node("researcher", researcher_executor)
graph.add_node("cot", cot_executor)
graph.add_node("math", math_executor)

graph.set_entry_point("plan")
graph.add_edge("plan", "router")

graph.add_conditional_edges("router", route, {
    "researcher": "researcher",
    "cot": "cot",
    "math": "math"
})

def is_done(state):
    return state["current_step"] >= len(state["tasks"])

graph.add_conditional_edges("researcher", is_done, {True: END, False: "router"})
graph.add_conditional_edges("cot", is_done, {True: END, False: "router"})
graph.add_conditional_edges("math", is_done, {True: END, False: "router"})

graph.set_finish_point(END)
app = graph.compile()

# ---------------- RUN ----------------
query = "Find Tesla's last quarter performance, get capital of France, and calculate revenue change from 10B to 12B."
result = app.invoke({"query": query})
print("\nMulti-Agent Routed Execution:")
for r in result["results"]:
    print(" -", r)


ImportError: cannot import name 'ToolExecutor' from 'langgraph.prebuilt' (/opt/conda/lib/python3.12/site-packages/langgraph/prebuilt/__init__.py)